In [2]:
import requests
import os
import json
#import csv
import pandas as pd 

import spacy
import geopy 
from geopy.extra.rate_limiter import RateLimiter
locator = geopy.geocoders.Nominatim(user_agent="mygeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=5)

nlp_wk = spacy.load("xx_ent_wiki_sm")

bearer_token = "AAAAAAAAAAAAAAAAAAAAAP1LRwEAAAAAn64BsczyRJiYHOVTTbN9415JH%2Fs%3DkzXwt5UPrYzTxL5sNVtqZD25ejB0UOtB4ayTcdZQfEUIwX26Qx"


In [3]:
def get_userTimeline(userDetails): 
    tweet_array = []
    for user_id in userDetails:
        url = "https://api.twitter.com/2/users/{}/tweets".format(user_id)
        payload={}
        headers = {
          'Authorization': 'Bearer '+bearer_token
        }
        response = requests.request("GET", url, headers=headers, data=payload)

        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line)
                #Prints loction for user's timeline
                
                for tweet in json_response['data']:
                    doc = nlp_wk(tweet['text'])
                    locations = []
                    locations.extend([ent.text for ent in doc.ents if ent.label_ in ['LOC']])
                    if(len(locations) > 0):
                        tweet_array.append([tweet['id'],tweet['text'],locations])
    tweetDatFrame = pd.DataFrame(tweet_array,columns=['author_id','text','loc'])
    
    tweetDatFrame = pd.DataFrame(tweet_array,columns=['author_id','text','loc'])
    tweetDatFrame.to_csv('data/userTimelineTweets.csv',sep=',', mode='a', index=False, encoding='utf-8',columns=['author_id','text','loc']) # Use Tab to seperate data            
    

In [4]:
def get_stream(): 
    query_melbRegions = ["melbourne"]
    
    with open('SA2Name_Melbourne.json') as f:
        melb_regions = json.load(f)
        for regions in melb_regions['features']:
            query_melbRegions.append(regions['properties']['sa2_name16'])

    n=10
    region_chunks = [query_melbRegions[i:i + n] for i in range(0, len(query_melbRegions), n)]
    tweet_array = []
    users_array = []

    for reg in region_chunks:
        
        reg_query = ' OR '.join(str(v) for v in reg)
        query = reg_query +" "+ "(beaches OR pubs OR stadiums OR Museu OR art Galleries OR Gardens OR Clubs OR Bars OR Restauraunt OR Hotels OR Motels OR Schools OR Universities OR Parlor OR Saloon OR Coffee Shop OR Casinos OR Hospitals)"

        tweetFields = "tweet.fields=created_at,lang"
        maxResults = "max_results=10"
        expansions = "expansions=author_id,geo.place_id"

        url = "https://api.twitter.com/2/tweets/search/recent?query="+query+"&"+tweetFields+"&"+maxResults+"&"+expansions+"&"

        payload={}
        headers = {
          'Authorization': 'Bearer '+bearer_token,
          'Cookie': 'guest_id=v1%3A162668876704133606; personalization_id="v1_QPzpEmtR+oVGPKUXrUpOIA=="'
        }

        response = requests.request("GET", url, headers=headers, data=payload)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(
                "Cannot get stream (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )    
        
        for response_line in response.iter_lines():
            if response_line:
                json_response = json.loads(response_line) 
                
                for tweet in json_response['data']:
                    doc = nlp_wk(tweet['text'])
                    locations = []
                    locations.extend([ent.text for ent in doc.ents if ent.label_ in ['LOC']])
                    if(len(locations) > 0):
                        tweet_array.append([tweet['author_id'],tweet['text'],locations])
                        users_array.append(tweet['author_id'])
    
            
    tweetDatFrame = pd.DataFrame(tweet_array,columns=['author_id','text','loc'])
    tweetDatFrame.to_csv('data/gatheredTweets.csv',sep=',', mode='a', index=False, encoding='utf-8',columns=['author_id','text','loc']) # Use Tab to seperate data            
    
    usersDatFrame = pd.DataFrame(users_array,columns=['author_id'])
    usersDatFrame.to_csv('data/userIds.csv',sep=',', mode='a', index=False, encoding='utf-8',columns=['author_id']) # Use Tab to seperate data            

    get_userTimeline(users_array)

In [ ]:
get_stream() 
import time
for i in range(1000):
    print('Doing hard work here i=' + str(i))
    get_stream() 
    print('Taking a nap now...')
    time.sleep(100)


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
Doing hard work here i=0
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
